# Step 1: Install Ollama (downloads and installs from the official script)
# This command fetches and executes the Ollama installation script
# using curl and a shell pipe.
# Make sure you have curl installed.
# Note: Run this in a shell environment (not directly in Python).
# ! is used to run shell commands in Jupyter cells.
!curl -fsSL https://ollama.com/install.sh | sh

# Step 2: Install additional system dependencies
# 'pciutils' and 'lshw' are useful for inspecting hardware (e.g., GPU availability)
!apt update && apt install -y pciutils lshw

# Step 3: Pull the 20B parameter open-source GPT model via Ollama
# This downloads the model locally so it can be run later.
!ollama pull gpt-oss:20b

# Step 4: Run the downloaded model
# This starts the model and lets you chat with it interactively in terminal.
!ollama run gpt-oss:20b


# Playing with the model using ollama endpoint and chat completions api

In [12]:
%pip install -q openai


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
from openai import OpenAI
 
client = OpenAI(
    base_url="http://localhost:11434/v1",  # Local Ollama API
    api_key="ollama"                       # Dummy key
)
 
response = client.chat.completions.create(
    model="gpt-oss:20b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant with gives short and crisp answers. Reasoning: low"},
        {"role": "user", "content": "Explain can AI takeover the humanity."}
    ]
)
 
print(response.choices[0].message.content)

**Short answer:**  
No. Current AI systems are narrow and lack the autonomy, motivation, and general intelligence required to “take over” humanity. Even a hypothetical future AGI would need human‑designed goals, safeguards, and alignment to act benignly—an unaligned, super‑intelligent system would be a theoretical risk, not a present reality.


# Detailed Response with chain of thought

In [15]:
from openai import OpenAI
import time
# Setup
client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

def chat_with_stats(model, messages):
    start_time = time.time()
    
    response = client.chat.completions.create(model=model, messages=messages)
    
    end_time = time.time()
    response_time = end_time - start_time
    usage = response.usage
    content = response.choices[0].message.content
    
    # Print stats
    print(f"🔢 Tokens: {usage.prompt_tokens} + {usage.completion_tokens} = {usage.total_tokens}")
    print(f"🤖 Model: {response.model}")
    print(f"✅ Status: {response.choices[0].finish_reason}")
    
    # Print reasoning if available
    message = response.choices[0].message
    reasoning = getattr(message, 'reasoning', None)
    if reasoning:
        print(f"🧠 Reasoning(Chain of Thought): {reasoning}")
    else:
        # Check if reasoning is in the raw response
        raw_dict = response.model_dump() if hasattr(response, 'model_dump') else response.__dict__
        try:
            reasoning_raw = raw_dict['choices'][0]['message'].get('reasoning')
            if reasoning_raw:
                print(f"🧠 Reasoning(Chain of Thought): {reasoning_raw}")
        except:
            print("🧠 Reasoning: Not available")
    
    print("-" * 50)
    print(content)
    
    return response

# Example usage
messages = [
    {"role": "system", "content": "You are a helpful assistant that answers in short Reasoning: low"},
    {"role": "user", "content": "Can AI Take over the world?"}
]

chat_with_stats("gpt-oss:20b", messages)

🔢 Tokens: 95 + 192 = 287
🤖 Model: gpt-oss:20b
✅ Status: stop
🧠 Reasoning(Chain of Thought): We need to answer within guidelines. Short reasoning: low. So just answer concisely with short reasoning note. Provide answer: No, can't take over world, technical, ethical constraints.
--------------------------------------------------
**Answer**: No – current AI systems lack the general intelligence, agency, and autonomous control necessary to seize power globally.

**Reasoning**:  
- AI is narrow, task‑specific, and requires human oversight for training, deployment, and updates.  
- Even advanced models don’t possess self‑awareness, motives, or the ability to coordinate large‑scale actions without human instructions.  
- Safeguards such as regulatory oversight, technical constraints (e.g., alignment, explainability), and legal frameworks are in place to prevent unintended misuse.  
- Ethical and societal norms, plus practical limits like hardware dependencies and data requirements, further in

ChatCompletion(id='chatcmpl-637', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Answer**: No – current AI systems lack the general intelligence, agency, and autonomous control necessary to seize power globally.\n\n**Reasoning**:  \n- AI is narrow, task‑specific, and requires human oversight for training, deployment, and updates.  \n- Even advanced models don’t possess self‑awareness, motives, or the ability to coordinate large‑scale actions without human instructions.  \n- Safeguards such as regulatory oversight, technical constraints (e.g., alignment, explainability), and legal frameworks are in place to prevent unintended misuse.  \n- Ethical and societal norms, plus practical limits like hardware dependencies and data requirements, further inhibit a single AI from autonomously “taking over” the world.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning="We need to answ

# Funtion Calling with gpt-oss

In [18]:
from openai import OpenAI

# Connect to Ollama's local API
client = OpenAI(
    base_url="http://localhost:11434/v1",  # Ollama API
    api_key="ollama"                       # Dummy key
)

# Define the function (tool) for getting weather
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather in a given Indian city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "enum": ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata"],
                        "description": "The name of the city to get weather for"
                    }
                },
                "required": ["city"]
            },
        },
    }
]

# Initial user message
messages = [
    {"role": "user", "content": "What's the weather like in Bangalore today?"}
]

# Call the model with the tool definition
response = client.chat.completions.create(
    model="gpt-oss:20b",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

# Extract the model's response
message = response.choices[0].message
print("Model response:", message)

# Step 2: Check if tool_call is requested
if message.tool_calls:
    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        args = eval(tool_call.function.arguments)

        # Dummy weather database
        weather_data = {
            "Mumbai": {"temp": "31°C", "condition": "Humid and Cloudy"},
            "Delhi": {"temp": "36°C", "condition": "Sunny with haze"},
            "Bangalore": {"temp": "27°C", "condition": "Cloudy with light showers"},
            "Chennai": {"temp": "33°C", "condition": "Hot and Sunny"},
            "Kolkata": {"temp": "30°C", "condition": "Thunderstorms likely"},
        }

        # Get the city from tool arguments
        city = args.get("city", "Unknown")

        # Generate response
        if city in weather_data:
            tool_result = f"The weather in {city} is {weather_data[city]['condition']} with a temperature of {weather_data[city]['temp']}."
        else:
            tool_result = f"Sorry, I don't have weather data for {city}."

        # Feed tool result back into the model for final response
        final_response = client.chat.completions.create(
            model="gpt-oss:20b",
            messages=[
                messages[0],  # original user message
                message,      # model's tool_call response
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": tool_name,
                    "content": tool_result
                }
            ]
        )

        print("\n✅ Final Answer:")
        print(final_response.choices[0].message.content)
else:
    print("No function call was made by the model.")


Model response: ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xf584efy', function=Function(arguments='{"city":"Bangalore"}', name='get_weather'), type='function', index=0)], reasoning='The user asks: "What\'s the weather like in Bangalore today?" We need to use the tool get_weather with city = Bangalore. Return the result.')

✅ Final Answer:
Here are the weather details for Bangalore today:

- **Condition:** Cloudy with light showers  
- **Temperature:** 27 °C  

Feel free to let me know if you’d like a forecast for the next few days or any other information!
